In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import pi
from scipy import special as sp
import scipy.integrate as integrate
from scipy.interpolate import CubicSpline
from tqdm import tqdm
from scipy.constants import fine_structure, speed_of_light, electron_mass, elementary_charge, hbar, epsilon_0
from scipy import optimize

rng = np.random.default_rng()

In [ ]:
#variáveis e constantes
electron_radius = elementary_charge**2/(4*pi*epsilon_0*electron_mass*speed_of_light**2)
num_pon = 10                                                    #número de pontos
mean_ene = 1800                                                    #energia média inicial
std_div = 90                                                       #desvio padrão
mag_field = np.array([2.5*10**6, 2.5*10**5, 2.5*10**4, 2.5*10**3]) #[T] intensidade do campo magnético
chi_0 = 1                                                          #valor de chi máximo
tau_e = hbar/(speed_of_light**2*electron_mass)                     #[s] time it takes for light to travel across the electron radius electron_radius/speed_of_light#
n_bins = 5                                                       #número de bins do histograma
t_end = np.array([3, 5, 20, 20])                                   #tempos finais
t_steps = 100                                                      #numero de steps

In [ ]:
#valores de gamma
gamma_glob = mean_ene + std_div*rng.standard_normal(num_pon)

#funcao para determinar o racio entre gamma e chi
def cnst_prop_2(mag):
    return mag/(4.5*10**(9))

def bessel_integral(v):
    return integrate.quad(lambda x: sp.kv(5/3, x), v, np.Inf)[0]

def G_tilde_int(chi, epsilon):
    v = 2*epsilon/(3*chi*(1-epsilon))
    int_smth = lambda v1, epsilon1, chi1: np.sqrt(3)/(2*pi)*epsilon1*(integrate.quad(lambda x: sp.kv(5/3, x), v1, np.Inf)[0] + 1.5*epsilon1*chi1*v1*sp.kv(2/3, v1))
    vec_int = np.vectorize(int_smth)
    return vec_int(v, epsilon, chi)

#determinar o rate de emissao de fotoes
def dN_dt(gamma, mag):
    minimo = np.min(gamma)
    omega_c = elementary_charge * mag / (electron_mass * mean_ene)
    int_smth_0 = lambda x, gamma1: G_tilde_int((gamma1*cnst_prop_2(mag)), x/gamma1)/(gamma1*x)
    int_smth = lambda gamma1: (2/3) * fine_structure/tau_e/omega_c*integrate.quad(int_smth_0, 0, minimo, args=(gamma1))[0]
    vec_int = np.vectorize(int_smth)
    return vec_int(gamma)

def integration(x1):
    return integrate.quad(lambda x: sp.kv(1 / 3, x), x1, np.Inf)[0]

chi_int = np.linspace(0, 1000, int(num_pon)) #chi_int = np.linspace(0, 1000, int(num_pon/10))

R = np.array([integration(x) for x in chi_int])

bessel_1_3_int = CubicSpline(chi_int, R)

#funcao de probabilidade de emissao de um fotao por um certo eletrao num dado intervalo de tempo
def d2P_dtdX(epsilon, mag, gamma):  # funcao analitica nao normalizada de compton scattering
    X_e = cnst_prop_2(mag) * gamma
    X_tilde = 2 * epsilon / (3 * X_e * (1 - epsilon))
    return 1/X_e * ((1 - epsilon + 1 / (1 - epsilon)) * sp.kv(2 / 3, X_tilde) - bessel_1_3_int(X_tilde))

#funcao acima normalizada
def d2P_dtdX_norm(epsilon, mag, gamma, integ):
    X_e = cnst_prop_2(mag) * gamma
    X_tilde = 2 * epsilon / (3 * X_e * (1 - epsilon))
    return (1 / X_e * ((1 - epsilon + 1 / (1 - epsilon)) * sp.kv(2 / 3,X_tilde) - bessel_1_3_int(X_tilde)))/integ

int_arr = np.linspace(0.0001, 9.999, num_pon)

#metodo de inverse transform sampling
def dP_dt(mag, gamma, integ, point):  # CDF
    return integrate.quad(lambda x: d2P_dtdX_norm(x, mag, gamma, integ), 0, point)[0]

def dP_dt_inverse_aux(y, x, mag, gamma, integ):
    return dP_dt(mag, gamma, integ, y) - x

def dP_dt_inverse(x, mag, gamma, integ):
    return optimize.bisect(dP_dt_inverse_aux, 0.0000001, 0.999999, args=(x, mag, gamma, integ))


In [ ]:
for i in tqdm(range(len(mag_field))):

    time_step = t_end[i]/t_steps

    gamma = np.array([gamma_glob, gamma_glob, gamma_glob, gamma_glob])

    gamma_2 = np.array([np.zeros(num_pon), np.zeros(num_pon), np.zeros(num_pon), np.zeros(num_pon)])

    for k in tqdm(range(t_steps)):
        
        #probilidade de um fotao libertar um fotao
        prob = dN_dt(gamma[i], mag_field[i])*time_step

        for j in range(num_pon):

            rnd_num_1 = rng.random()
            
            #verificar se o fotao é libertado
            if prob[j] < rnd_num_1:

                N_d2P_dt_dX_arr = np.array([d2P_dtdX(int_arr, mag_field[i], gamma[i][j])])

                integ = np.trapz(int_arr, N_d2P_dt_dX_arr)

                x_uniform = np.linspace(0.000001, 0.9999, 10)

                dP_dt_ICDF = np.array([dP_dt_inverse(x, mag_field[i], gamma[i][j], integ) for x in x_uniform])  # IDCF

                func = CubicSpline(x_uniform, dP_dt_ICDF)

                uni_rnd = rng.random()
                
                #subtrair a energia do fotão libertado
                gamma[i][j] *= (1 - func(uni_rnd))

        if (k == int(t_steps/2 - 1)):
            gamma_2[i] = gamma[i]

In [ ]:
#gerar os gráficos
fig, axis = plt.subplots(2, 2)

fig.suptitle('Distribution Evolution of Electron Beams')

fig.tight_layout(pad=2.8)

for i in tqdm(range(len(mag_field))):
    if (i == 3):
        # histograma em t=0
        axis[0, 0].hist(gamma_glob / mean_ene, n_bins, density=True,
                        color='red', alpha=0.5,
                        histtype='bar')

        # histograma em t = tend/2
        axis[0, 0].hist(gamma_2[0] / mean_ene, n_bins, density=True,
                        color='green', alpha=0.5,
                        histtype='bar')

        # histograma em t = tend
        axis[0, 0].hist(gamma[0] / mean_ene, n_bins, density=True,
                        color='blue', alpha=0.5,
                        histtype='bar')

    if (i == 2):
        # histograma em t=0
        axis[0, 1].hist(gamma_glob / mean_ene, n_bins, density=True,
                        color='red', alpha=0.5,
                        histtype='bar')

        # histograma em t = tend/2
        axis[0, 1].hist(
            gamma_2[1] / mean_ene,
            n_bins, density=True,
            color='green', alpha=0.5,
            histtype='bar')

        # histograma em t = tend
        axis[0, 1].hist(
            gamma[1] / mean_ene,
            n_bins, density=True,
            color='blue', alpha=0.5,
            histtype='bar')

    if (i == 1):
        # histograma em t=0
        axis[1, 0].hist(gamma_glob / mean_ene, n_bins, density=True,
                        color='red', alpha=0.5,
                        histtype='bar')

        # histograma em t = tend/2
        axis[1, 0].hist(
            gamma_2[2] / mean_ene,
            n_bins, density=True,
            color='green', alpha=0.5,
            histtype='bar')

        # histograma em t = tend
        axis[1, 0].hist(
            gamma[2] / mean_ene,
            n_bins, density=True,
            color='blue', alpha=0.5,
            histtype='bar')

    if (i == 0):
        # histograma em t=0
        axis[1, 1].hist(gamma_glob / mean_ene, n_bins, density=True,
                        color='red', alpha=0.5,
                        histtype='bar')

        # histograma em t = tend/2
        axis[1, 1].hist(
            gamma_2[3] / mean_ene,
            n_bins, density=True,
            color='green', alpha=0.5,
            histtype='bar')

        # histograma em t = tend
        axis[1, 1].hist(
            gamma[3] / mean_ene,
            n_bins, density=True,
            color='blue', alpha=0.5,
            histtype='bar')

plt.show()